In [60]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from string import punctuation
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, SimpleRNN, LSTM, GRU, Bidirectional, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [3]:
data = pd.read_table('ratings_total.txt', names = ['ratings', 'reviews'])
data

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
199995,2,장마라그런가!!! 달지않아요
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,5,넘이쁘고 쎄련되보이네요~


In [53]:
np.unique(data['ratings'])

array([1, 2, 4, 5], dtype=int64)

In [7]:
data.isna().sum()

ratings    0
reviews    0
dtype: int64

In [9]:
data.reviews.nunique()

199908

In [14]:
data = data.drop_duplicates(subset = ['reviews'])
data

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
199995,2,장마라그런가!!! 달지않아요
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,5,넘이쁘고 쎄련되보이네요~


In [20]:
stop_w = ['은', '는', '이', '가', '을', '를', '도', '것', '에', '에게', '로', '고', '요', '나', '저', '우리', '에서', '저희']
pc_k = list(punctuation)
stop_w + pc_k

['은',
 '는',
 '이',
 '가',
 '을',
 '를',
 '도',
 '것',
 '에',
 '에게',
 '로',
 '고',
 '요',
 '나',
 '저',
 '우리',
 '에서',
 '저희',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [19]:
data.reviews = data.reviews.str.replace('[^ㄱ-ㅎ, ㅏ-ㅣ 가-힣]', '')
data.reviews = data.reviews.str.replace('^ +', '')
data.reviews = data.reviews.replace('', np.nan)
data = data.dropna(how = 'any', axis = 0)
data

C:\Users\김철용\AppData\Local\Temp/ipykernel_13208/2602935626.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data.reviews = data.reviews.str.replace('[^ㄱ-ㅎ, ㅏ-ㅣ 가-힣]', '')
C:\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\김철용\AppData\Local\Temp/ipykernel_13208/2602935626.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.reviews = data.reviews.str.replace('^ +', '')


,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서개 더 구매했어요 이가격에 대박입니다 바느질이 조금 엉성...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...
4,5,민트색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
199995,2,장마라그런가 달지않아요
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,5,넘이쁘고 쎄련되보이네요


In [21]:
otk = Okt()
ot_l = []
for review in data['reviews']:
    ot_l.append(otk.morphs(review))
ot_l

[['배공', '빠르고', '굿'],
 ['택배',
  '가',
  '엉망',
  '이네',
  '용',
  '저희',
  '집',
  '밑',
  '에',
  '층',
  '에',
  '말',
  '도',
  '없이',
  '놔두고가고'],
 ['아주',
  '좋아요',
  '바지',
  '정말',
  '좋아서',
  '개',
  '더',
  '구매',
  '했어요',
  '이',
  '가격',
  '에',
  '대박',
  '입니다',
  '바느질',
  '이',
  '조금',
  '엉',
  '성하긴',
  '하지만',
  '편하고',
  '가성',
  '비',
  '최고',
  '예요'],
 ['선물',
  '용',
  '으로',
  '빨리',
  '받아서',
  '전달',
  '했어야',
  '하는',
  '상품',
  '이었는데',
  '머그컵',
  '만',
  '와서',
  '당황',
  '했습니다',
  '전화했더니',
  '바로',
  '주신다',
  '했지만',
  '배송',
  '도',
  '누락',
  '되어있었네요',
  '확인',
  '안',
  '하고',
  '바로',
  '선물',
  '했으면',
  '큰일',
  '날',
  '뻔했네요',
  '이렇게',
  '배송',
  '이',
  '오래',
  '걸렸으면',
  '사는거',
  '다시',
  '생각',
  '했을거',
  '같아요',
  '아쉽네요'],
 ['민트',
  '색상',
  '예뻐요',
  '옆',
  '손잡이',
  '는',
  '거',
  '는',
  '용',
  '도로',
  '도',
  '사용',
  '되네요',
  'ㅎㅎ'],
 ['비추',
  '합니다',
  '계란',
  '뒤집을',
  '때',
  '완전',
  '불편해요',
  'ㅠㅠ',
  '코팅',
  '도',
  '묻어나고',
  '보기',
  '엔',
  '예쁘고',
  '실용',
  '적',
  '으로',
  '보였는데',
  '생각',
  '보다',
  '진짜',
  '별로',
  '입니

In [24]:
st_l = []
for tk in ot_l:
    st_l.append([word for word in tk if word not in stop_w and len(word) > 1])
st_l

[['배공', '빠르고'],
 ['택배', '엉망', '이네', '없이', '놔두고가고'],
 ['아주',
  '좋아요',
  '바지',
  '정말',
  '좋아서',
  '구매',
  '했어요',
  '가격',
  '대박',
  '입니다',
  '바느질',
  '조금',
  '성하긴',
  '하지만',
  '편하고',
  '가성',
  '최고',
  '예요'],
 ['선물',
  '으로',
  '빨리',
  '받아서',
  '전달',
  '했어야',
  '하는',
  '상품',
  '이었는데',
  '머그컵',
  '와서',
  '당황',
  '했습니다',
  '전화했더니',
  '바로',
  '주신다',
  '했지만',
  '배송',
  '누락',
  '되어있었네요',
  '확인',
  '하고',
  '바로',
  '선물',
  '했으면',
  '큰일',
  '뻔했네요',
  '이렇게',
  '배송',
  '오래',
  '걸렸으면',
  '사는거',
  '다시',
  '생각',
  '했을거',
  '같아요',
  '아쉽네요'],
 ['민트', '색상', '예뻐요', '손잡이', '도로', '사용', '되네요', 'ㅎㅎ'],
 ['비추',
  '합니다',
  '계란',
  '뒤집을',
  '완전',
  '불편해요',
  'ㅠㅠ',
  '코팅',
  '묻어나고',
  '보기',
  '예쁘고',
  '실용',
  '으로',
  '보였는데',
  '생각',
  '보다',
  '진짜',
  '별로',
  '입니다'],
 ['주문',
  '시켰는데',
  '월일',
  '배송',
  '왔네요',
  'ㅎㅎㅎ',
  '여기',
  '회사',
  '과는',
  '전화',
  '안되고',
  '아무런',
  '연락',
  '받을수가',
  '없으니',
  '답답하신',
  '다른',
  '사시는거',
  '추천',
  '드립니다'],
 ['넉넉한', '길이', '주문', '했는데도', '맞네요', '별로', '예요'],
 ['보폴', '계속', '처럼', '나오다가', '

In [25]:
t_l = []
for sentence in st_l:
    t_l.append([' '.join(sentence)])
t_l

[['배공 빠르고'],
 ['택배 엉망 이네 없이 놔두고가고'],
 ['아주 좋아요 바지 정말 좋아서 구매 했어요 가격 대박 입니다 바느질 조금 성하긴 하지만 편하고 가성 최고 예요'],
 ['선물 으로 빨리 받아서 전달 했어야 하는 상품 이었는데 머그컵 와서 당황 했습니다 전화했더니 바로 주신다 했지만 배송 누락 되어있었네요 확인 하고 바로 선물 했으면 큰일 뻔했네요 이렇게 배송 오래 걸렸으면 사는거 다시 생각 했을거 같아요 아쉽네요'],
 ['민트 색상 예뻐요 손잡이 도로 사용 되네요 ㅎㅎ'],
 ['비추 합니다 계란 뒤집을 완전 불편해요 ㅠㅠ 코팅 묻어나고 보기 예쁘고 실용 으로 보였는데 생각 보다 진짜 별로 입니다'],
 ['주문 시켰는데 월일 배송 왔네요 ㅎㅎㅎ 여기 회사 과는 전화 안되고 아무런 연락 받을수가 없으니 답답하신 다른 사시는거 추천 드립니다'],
 ['넉넉한 길이 주문 했는데도 맞네요 별로 예요'],
 ['보폴 계속 처럼 나오다가 지금 안나'],
 ['인데 전문 속옷 브랜드 위생 팬티 보다 작은듯 불편해요 밴딩부분 신축 없는 일반 실로 되어있어 빅사 에도 빅사 이즈 같지 않아요 입고 편하게 밴딩부분 늘어나고 입었을 밴딩 잡아주어야하는데'],
 ['사이즈 딱이고 귀엽고 좋아요 ㅎㅎ'],
 ['베이지 구매 했는데 약간 살색 가까워요'],
 ['화면 인가 봐요 노란 컬러 돋보여요 저렴한 그냥 씁니다'],
 ['별루 ㅏㅛㅇ 탛캏타 ㅗㅗㅗ ㅕㅐㅛㅌ ㅔㅎ'],
 ['촉감 좋고 무게 이나 편합니다'],
 ['멍하기 좋고 사이즈 너무 좋아요'],
 ['재구매 친구 좋은 난다고'],
 ['실내 신는 건지 몰랐어요'],
 ['재구매 좋은데 하나 이상하네요'],
 ['가게 운영 하는 사장 께서 대신 구매 해달라고 하셔서 구매 하게 되었는데 가게 달아놓으니까 이쁘네요'],
 ['가성 괜찮습니다 바퀴 고정 된다면 다섯 린데'],
 ['화장 깨끗하게 지워지지 않아요 거품 생기 오랫동안 문질러서 씻어내야하지만 정력 별로 듯해'],
 ['남자 인데 너무 좋아요 유해

In [30]:
tk = Tokenizer()
tk.fit_on_texts(st_l)
len(tk.word_index)

108607

In [31]:
ck1 = list(tk.word_counts.items())
ck2 = sorted(ck1, key = lambda x: x[1])
ck3 = [i for i in ck2 if i[1] > 5]
len(ck3)

20194

In [32]:
tk = Tokenizer(num_words = 20195)
tk.fit_on_texts(st_l)

In [33]:
s_tk = tk.texts_to_sequences(st_l)
s_tk

[[27],
 [107, 354, 73, 79],
 [38, 3, 336, 29, 141, 5, 28, 11, 697, 16, 863, 44, 185, 143, 88, 135, 84],
 [63,
  7,
  163,
  532,
  3283,
  4200,
  108,
  25,
  1651,
  14132,
  151,
  636,
  87,
  3563,
  90,
  18243,
  257,
  2,
  1153,
  131,
  14,
  90,
  63,
  1537,
  3242,
  4201,
  168,
  2,
  114,
  6988,
  61,
  13,
  14133,
  15,
  165],
 [2338, 71, 157, 363, 2210, 6, 208, 35],
 [233,
  24,
  1603,
  51,
  295,
  22,
  986,
  8426,
  472,
  291,
  1025,
  7,
  3428,
  13,
  12,
  39,
  19,
  16],
 [17,
  196,
  2046,
  2,
  99,
  297,
  121,
  594,
  1523,
  302,
  309,
  2359,
  220,
  2943,
  30,
  16579,
  68,
  764],
 [2971, 174, 17, 1039, 501, 19, 84],
 [16580, 78, 140, 18244, 262, 255],
 [34,
  7634,
  1026,
  938,
  1388,
  637,
  12,
  2757,
  295,
  12404,
  815,
  308,
  427,
  6819,
  2831,
  5462,
  116,
  5462,
  506,
  1801,
  109,
  376,
  327,
  12404,
  3164,
  3009,
  5787],
 [21, 4251, 923, 3, 35],
 [1346, 5, 10, 153, 6820, 6341],
 [270, 763, 275, 3388, 549,

In [76]:
np.unique(s_tk)

C:\anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


array([list([]), list([1]), list([1, 1]), ...,
       list([20192, 2915, 274, 8, 6360, 429, 957, 161, 562]),
       list([20192, 7317, 411]), list([20193, 404, 4180, 68])],
      dtype=object)

In [77]:
n_idx = [i for i, n in enumerate(st_l) if len(n) < 1]
n_idx

[1008,
 2867,
 3328,
 4444,
 5276,
 5829,
 6849,
 8770,
 10390,
 11797,
 12993,
 14199,
 14829,
 15253,
 15415,
 16677,
 19241,
 19863,
 22316,
 26701,
 26878,
 30311,
 32956,
 33580,
 33653,
 37163,
 38341,
 43707,
 44627,
 45777,
 47908,
 47928,
 49398,
 51099,
 51579,
 53957,
 54669,
 58222,
 58480,
 59680,
 60614,
 61219,
 61677,
 62182,
 62902,
 63796,
 67897,
 68555,
 69312,
 70545,
 71665,
 72680,
 73786,
 74346,
 83100,
 83973,
 84681,
 85613,
 85978,
 86100,
 88556,
 88977,
 93079,
 93122,
 95051,
 102229,
 102275,
 104639,
 105709,
 105934,
 109198,
 111125,
 111897,
 114750,
 116030,
 119505,
 120548,
 121765,
 121850,
 123346,
 123440,
 123457,
 125784,
 126053,
 127185,
 129441,
 129887,
 132238,
 135834,
 135941,
 135987,
 139194,
 140725,
 141440,
 148602,
 149005,
 151155,
 152034,
 152470,
 152587,
 152882,
 152885,
 154641,
 157844,
 158535,
 161793,
 162360,
 163061,
 164208,
 166844,
 173111,
 173125,
 174294,
 177439,
 179535,
 183311,
 184378,
 186679,
 187445,
 1

In [103]:
X_data = s_tk
Y_data = np.array(data['ratings'].values)

In [104]:
X_data = np.delete(X_data, n_idx, axis = 0)
Y_data = np.delete(Y_data, n_idx, axis = 0)

C:\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [105]:
len(X_data), len(Y_data)

(199784, 199784)

In [111]:
np.unique(Y_data)

array([1, 2, 4, 5], dtype=int64)

In [116]:
Y_data = to_categorical(Y_data)
Y_data

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [117]:
p_x_data = pad_sequences(X_data)

In [118]:
x_data, tt_x, y_data, tt_y = train_test_split(p_x_data, Y_data, test_size = 0.3, random_state = 22)
t_x, v_x, t_y, v_y = train_test_split(x_data, y_data, test_size = 0.2, random_state = 22)

In [119]:
t_x.shape, t_y.shape, v_x.shape, v_y.shape

((111878, 45), (111878, 6), (27970, 45), (27970, 6))

In [120]:
batch_size = 128
total_word_num = 20195
max_sentence_len = 45
embedding_len = 128
neuron_num = 128
d_out = 0.6
lr = 0.001
ec = 5

In [123]:
m1 = Sequential()
m1.add(Embedding(total_word_num, embedding_len, input_length = max_sentence_len))
m1.add(SimpleRNN(neuron_num, dropout = d_out, return_sequences = True))
m1.add(SimpleRNN(neuron_num, dropout = d_out))
m1.add(Dense(neuron_num, activation = 'relu'))
m1.add(Dropout(d_out))
m1.add(BatchNormalization())
m1.add(Dense(neuron_num, activation = 'relu'))
m1.add(Dropout(d_out))
m1.add(BatchNormalization())
m1.add(Dense(6, activation = 'softmax'))
m1.compile(optimizer = Adam(lr), loss = tf.losses.CategoricalCrossentropy(), metrics = ['acc'])
e_stopping = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 3, restore_best_weights = True)

In [124]:
hy1 = m1.fit(t_x, t_y, epochs = ec, validation_data = (v_x, v_y), callbacks = [e_stopping])

Epoch 1/5
3497/3497 [==============================] - 198s 56ms/step - loss: 1.3358 - acc: 0.3815 - val_loss: 1.2658 - val_acc: 0.4060
Epoch 2/5
3497/3497 [==============================] - 198s 57ms/step - loss: 1.2692 - acc: 0.3987 - val_loss: 1.2670 - val_acc: 0.4060
Epoch 3/5
3497/3497 [==============================] - 210s 60ms/step - loss: 1.2667 - acc: 0.4010 - val_loss: 1.2690 - val_acc: 0.4060
Epoch 4/5
3497/3497 [==============================] - 199s 57ms/step - loss: 1.2652 - acc: 0.4029 - val_loss: 1.2671 - val_acc: 0.4060


In [125]:
m2 = Sequential()
m2.add(Embedding(total_word_num, embedding_len, input_length = max_sentence_len))
m2.add(LSTM(neuron_num, dropout = d_out, return_sequences = True))
m2.add(LSTM(neuron_num, dropout = d_out))
m2.add(Dense(neuron_num, activation = 'relu'))
m2.add(Dropout(d_out))
m2.add(BatchNormalization())
m2.add(Dense(neuron_num, activation = 'relu'))
m2.add(Dropout(d_out))
m2.add(BatchNormalization())
m2.add(Dense(6, activation = 'softmax'))
m2.compile(optimizer = Adam(lr), loss = tf.losses.CategoricalCrossentropy(), metrics = ['acc'])
e_stopping = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 3, restore_best_weights = True)

In [126]:
hy2 = m2.fit(t_x, t_y, epochs = ec, validation_data = (v_x, v_y), callbacks = [e_stopping])

Epoch 1/5
3497/3497 [==============================] - 381s 108ms/step - loss: 1.0133 - acc: 0.5874 - val_loss: 0.8392 - val_acc: 0.6498
Epoch 2/5
3497/3497 [==============================] - 359s 103ms/step - loss: 0.8329 - acc: 0.6553 - val_loss: 0.8230 - val_acc: 0.6501
Epoch 3/5
3497/3497 [==============================] - 363s 104ms/step - loss: 0.7968 - acc: 0.6678 - val_loss: 0.7998 - val_acc: 0.6635
Epoch 4/5
3497/3497 [==============================] - 370s 106ms/step - loss: 0.7648 - acc: 0.6849 - val_loss: 0.8078 - val_acc: 0.6623
Epoch 5/5
3497/3497 [==============================] - 378s 108ms/step - loss: 0.7398 - acc: 0.7000 - val_loss: 0.8340 - val_acc: 0.6558


In [127]:
m3 = Sequential()
m3.add(Embedding(total_word_num, embedding_len, input_length = max_sentence_len))
m3.add(GRU(neuron_num, dropout = d_out, return_sequences = True))
m3.add(GRU(neuron_num, dropout = d_out))
m3.add(Dense(neuron_num, activation = 'relu'))
m3.add(Dropout(d_out))
m3.add(BatchNormalization())
m3.add(Dense(neuron_num, activation = 'relu'))
m3.add(Dropout(d_out))
m3.add(BatchNormalization())
m3.add(Dense(6, activation = 'softmax'))
m3.compile(optimizer = Adam(lr), loss = tf.losses.CategoricalCrossentropy(), metrics = ['acc'])
e_stopping = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 3, restore_best_weights = True)

In [128]:
hy3 = m3.fit(t_x, t_y, epochs = ec, validation_data = (v_x, v_y), callbacks = [e_stopping])

Epoch 1/5
3497/3497 [==============================] - 332s 94ms/step - loss: 0.9915 - acc: 0.5994 - val_loss: 0.8336 - val_acc: 0.6506
Epoch 2/5
3497/3497 [==============================] - 319s 91ms/step - loss: 0.8355 - acc: 0.6546 - val_loss: 0.8265 - val_acc: 0.6481
Epoch 3/5
3497/3497 [==============================] - 314s 90ms/step - loss: 0.8018 - acc: 0.6648 - val_loss: 0.8180 - val_acc: 0.6563
Epoch 4/5
3497/3497 [==============================] - 315s 90ms/step - loss: 0.7736 - acc: 0.6800 - val_loss: 0.8152 - val_acc: 0.6581
Epoch 5/5
3497/3497 [==============================] - 350s 100ms/step - loss: 0.7482 - acc: 0.6952 - val_loss: 0.8178 - val_acc: 0.6640


In [130]:
m4 = Sequential()
m4.add(Embedding(total_word_num, embedding_len, input_length = max_sentence_len))
m4.add(Bidirectional(LSTM(neuron_num, dropout = d_out, return_sequences = True)))
m4.add(Bidirectional(LSTM(neuron_num, dropout = d_out)))
m4.add(Dense(neuron_num, activation = 'relu'))
m4.add(Dropout(d_out))
m4.add(BatchNormalization())
m4.add(Dense(neuron_num, activation = 'relu'))
m4.add(Dropout(d_out))
m4.add(BatchNormalization())
m4.add(Dense(6, activation = 'softmax'))
m4.compile(optimizer = Adam(lr), loss = tf.losses.CategoricalCrossentropy(), metrics = ['acc'])
e_stopping = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 3, restore_best_weights = True)

In [131]:
hy4 = m4.fit(t_x, t_y, epochs = ec, validation_data = (v_x, v_y), callbacks = [e_stopping])

Epoch 1/5
3497/3497 [==============================] - 699s 199ms/step - loss: 1.0239 - acc: 0.5818 - val_loss: 0.8494 - val_acc: 0.6468
Epoch 2/5
3497/3497 [==============================] - 741s 212ms/step - loss: 0.8329 - acc: 0.6570 - val_loss: 0.8195 - val_acc: 0.6502
Epoch 3/5
3497/3497 [==============================] - 761s 218ms/step - loss: 0.7994 - acc: 0.6674 - val_loss: 0.8353 - val_acc: 0.6506
Epoch 4/5
3497/3497 [==============================] - 755s 216ms/step - loss: 0.7729 - acc: 0.6750 - val_loss: 0.8340 - val_acc: 0.6472
Epoch 5/5
3497/3497 [==============================] - 791s 226ms/step - loss: 0.7553 - acc: 0.6817 - val_loss: 0.8554 - val_acc: 0.6533


In [132]:
m1.evaluate(tt_x, tt_y)

1873/1873 [==============================] - 17s 9ms/step - loss: 1.2596 - acc: 0.4052


[1.2596309185028076, 0.4052155613899231]

In [133]:
m2.evaluate(tt_x, tt_y)

1873/1873 [==============================] - 59s 31ms/step - loss: 0.8276 - acc: 0.6617


[0.8275768756866455, 0.6616724729537964]

In [134]:
m3.evaluate(tt_x, tt_y)

1873/1873 [==============================] - 45s 24ms/step - loss: 0.8082 - acc: 0.6702


[0.808164656162262, 0.67024827003479]

In [135]:
m4.evaluate(tt_x, tt_y)

1873/1873 [==============================] - 114s 60ms/step - loss: 0.8095 - acc: 0.6596


[0.8095173835754395, 0.6595535278320312]